In [ ]:
#For Kaggle
#date=''

#For WL data
source='WL'
date='20241118'

saveImage=False

packageName='06-sklearn.tree'
classifierName='DecisionTreeClassifier'
extraParameterName='5-Random overundersampling'

print('done')

In [ ]:
import sys
import os
from importlib import reload
fpath = os.path.join('..//scripts')
sys.path.append(fpath)

import warnings
warnings.filterwarnings('ignore')

#loading internal scripts
import frauddetection as fd
import sourcedata as sd
import dataimport as di
import result as resultMd
import sampling as sp
reload(fd)
reload(sd)
reload(di)
reload(resultMd)
reload(sp)

print('done')

In [ ]:
import pandas as pd

dfTrx = pd.read_csv('../data/cleaned/'+source+'export'+date+'.csv')
dfTrx.head(5)

In [ ]:
# Hyperparameters Results
# not too high to avoid overfitting (lower or equals to 12 ?)
max_depthFound=18
min_samples_leafFound=7
min_samples_splitFound=17
criterionFound='entropy'

resultMd.update_hyperparameter_config_result(packageName,classifierName,extraParameterName,max_depthFound)

print('done')

In [ ]:
from sklearn.model_selection import train_test_split

predictors = fd.getPredictors(dfTrx)
TEST_SIZE = 0.20 # test size using_train_test_split
RANDOM_STATE = 0
x_train0, x_test, y_train0, y_test = train_test_split(dfTrx[predictors], dfTrx['Class'], test_size = TEST_SIZE, 
                                                        stratify= dfTrx['Class'],
                                                        random_state = RANDOM_STATE)

x_train, y_train = sp.randomOverUnderSampling(x_train0, y_train0)


In [ ]:
%%script false
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import randint

modelClf = DecisionTreeClassifier(random_state=42)
dic_param={
    'criterion':["gini","entropy"],
    'max_depth': randint(9,17),
    'min_samples_leaf': randint(8,25),
    'min_samples_split': randint(14,25),
}


res=fd.hyperparameterSelectionRandomizedSearchCVSampling(modelClf, dic_param, 'f1', x_train, y_train)
print(res)

#{'criterion': 'entropy', 'max_depth': 13, 'min_samples_leaf': 15, 'min_samples_split': 21}
#0.6215351320214746
#score   0.6963752665245203

#{'criterion': 'entropy', 'max_depth': 16, 'min_samples_leaf': 9, 'min_samples_split': 19}
#0.694422405989343
#score   0.7920498368436666

In [ ]:
%%script false
from sklearn.tree import DecisionTreeClassifier

modelClf = DecisionTreeClassifier(random_state=42)
# starting point n_estimators=100,  max_depth=7 
dic_param={
    'criterion':["gini","entropy"],
    'max_depth': [15,16,17,18],
    'min_samples_leaf': [7,8,9,10],
    'min_samples_split': [17,18,19,20]
}




res=fd.hyperparameterSelectionGridSearchCVSampling(modelClf, dic_param, 'f1', x_train, y_train)
print(res)

#{'criterion': 'entropy', 'max_depth': 16, 'min_samples_leaf': 15, 'min_samples_split': 14}
#0.6579876488241638
#scoref1 0.7208689233364969

#{'criterion': 'entropy', 'max_depth': 14, 'min_samples_leaf': 14, 'min_samples_split': 20}
#0.6459797587844633
#scoref1 0.7212121212121212

#{'criterion': 'entropy', 'max_depth': 17, 'min_samples_leaf': 8, 'min_samples_split': 18}
#0.7141612058115839
#scoref1 0.808695652173913

#{'criterion': 'entropy', 'max_depth': 18, 'min_samples_leaf': 7, 'min_samples_split': 17}
#0.7363769702193709
#scoref1 0.8294588786872436

In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import RandomOverSampler




modelClf = DecisionTreeClassifier(random_state=42)
parameters={'criterion': criterionFound, 'max_depth': max_depthFound, 
            'min_samples_leaf': min_samples_leafFound, 'min_samples_split': min_samples_splitFound}
modelClf.set_params(**parameters)

modelClf.fit(x_train, y_train)

predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

fd.print_scores(y_train, predsTrain,'f1', False)
fd.print_scores(y_test, predsTest,'f1')
fd.show_importance(modelClf, predictors)
fd.show_confusion_matrix(y_test, predsTest)
fd.show_prediction_graph(modelClf, x_test,y_test)


In [ ]:
files = fd.getAllFiles()

range = []
f1s = []
rocs = []
loop =0
for file in files:
    loop=loop+1
    range.append(loop)
    print(file)
  
    dfTrx = pd.read_csv('../data/cleaned/'+source+file)
    predictors = fd.getPredictors(dfTrx)

    preds = modelClf.predict(dfTrx[predictors])

    f1,mcc,roc= fd.print_scores(dfTrx['Class'], preds,'All', True)
    #fd.show_importance(modelClf,predictors)
    fd.show_confusion_matrix(dfTrx['Class'], preds)
    f1s.append(f1)
    rocs.append(roc)

fd.plt_train_test(range, f1s)

resultMd.update_performance_nextdays_result(packageName,classifierName,extraParameterName, f1s[0],f1s[1],f1s[2],f1s[3],rocs[0],rocs[1],rocs[2],rocs[3])

